In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/My Drive/Data/")

Loading the data:

In [ ]:
# load data again
import numpy as np
import pickle

with open("multiomics.pickle", "rb") as file:
    data_multiomics = pickle.load(file)

# display the column names and their respective counts
data_multiomics.columns.get_level_values(0).value_counts()

cellfree_rna               37275
microbiome                 18548
metabolomics                3485
plasma_somalogic            1300
immune_system                534
plasma_luminex                62
serum_luminex                 62
Training/Validation            1
Gates ID                       1
MRN                            1
Study Subject ID Number        1
Sex                            1
sex_bin                        1
timepoint                      1
gestational_age                1
dtype: int64

Transforming the input into:
1. Vanilla - keeps the features intact for training
2. Individual PCA - PCA on each feature individually keeping 95% of the variance in the feature-space
3. Whole PCA - PCA on all features combined keeping 95% of the variance in the feature-space

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler

# Input variable names
input_variable_names = ["cellfree_rna", "microbiome", "metabolomics",
                        "plasma_somalogic", "immune_system", "plasma_luminex", "serum_luminex"]

# variance percentage
variance_percentage = 0.95
pca = PCA(n_components=variance_percentage)

# Dictionary to store input variables
inputs = {}

# Dictionary to store input variables with PCA
input_variables_pca = {}

# Perform PCA for each input variable
for variable in input_variable_names:
    # Get the data for the variable
    data = data_multiomics[variable]

    # Store the input variable
    inputs[variable] = data

    # Perform PCA with 95% variance
    transformed_data = pca.fit_transform(data)

    # Store the PCA-transformed data
    input_variables_pca[variable] = transformed_data

# Concatenate the individual input variables
X_vanilla = np.hstack([inputs[variable]
                         for variable in input_variable_names])

# Concatenate the individual PCA-transformed data
X_individual_PCA = np.hstack([input_variables_pca[variable]
                             for variable in input_variable_names])

# Normalize the stacked inputs using StandardScaler
scaler = StandardScaler()
normalized_inputs = scaler.fit_transform(X_vanilla)

# Perform PCA to reduce dimensionality preserving 95% variance
X_whole_PCA = pca.fit_transform(normalized_inputs)

# Display the shapes of the input data
print("X_vanilla shape:", X_vanilla.shape)
print("X_individual_PCA shape:", X_individual_PCA.shape)
print("X_whole_PCA shape:", X_whole_PCA.shape)

X_vanilla shape: (68, 61266)
X_individual_PCA shape: (68, 103)
X_whole_PCA shape: (68, 51)


CNN implementation. I started out with more factors in the hyperparameters but my Google Colab resources protested hard against how much work they would do for free. So I had to make some tough calls.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import itertools

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, hidden_size, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.conv1(x.squeeze(2))
        out = self.relu(out)
        out = torch.mean(out, dim=2)
        out = self.fc(out)
        return out

# Set the number of bootstrap iterations
n_bootstrap_iterations = 100

# Prepare the data
# the X's have been already generated in the previous step
y = data_multiomics["Sex"].map({"Female": 0, "Male": 1}).to_numpy()

# Convert data to PyTorch tensors
X_vanilla = torch.Tensor(X_vanilla)
X_individual_PCA = torch.Tensor(X_individual_PCA)
X_whole_PCA = torch.Tensor(X_whole_PCA)
y = torch.Tensor(y)

# Define the hyperparameters and their values for grid search
hyperparameters = {
    'learning_rate': [0.1],
    'hidden_size': [32],
    'batch_size': [16, 32],
    'kernel_size': [3, 7],
    'pooling': ['max', 'avg'],
    'dropout_rate': [0.1],
    'weight_decay': [0.01],
    'optimizer': ['adam', 'sgd'],
    'learning_rate_schedule': ['fixed', 'step_decay'],
    'network_depth': [2, 4],
    'X_choice': ['vanilla', 'individual_PCA', 'whole_PCA']
}

# Generate all possible combinations of hyperparameters
combinations = list(itertools.product(*hyperparameters.values()))

# Initialize variables to store the best combination and accuracy
best_combination = None
best_accuracy = 0.0

# Check if CUDA is available and move the model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Iterate over each combination
for combination in tqdm(combinations, desc="Grid Search Progress"):
    learning_rate, hidden_size, batch_size, kernel_size, pooling, dropout_rate, weight_decay, optimizer, learning_rate_schedule, network_depth, X_choice = combination

    # Select the appropriate X based on the choice
    if X_choice == 'vanilla':
        X = X_vanilla
    elif X_choice == 'individual_PCA':
        X = X_individual_PCA
    elif X_choice == 'whole_PCA':
        X = X_whole_PCA

    # Create an instance of the CNN model
    model = CNN(X.shape[1], hidden_size, 2).to(device)

    # Define the optimizer and loss function
    criterion = nn.CrossEntropyLoss()
    if optimizer == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer == 'rmsprop':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # Perform bootstrapping
    accuracy_scores = []
    for _ in range(n_bootstrap_iterations):
        # Resample the data
        X_boot, y_boot = resample(X, y)

        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(
            X_boot, y_boot, test_size=0.2, stratify=y_boot)

        # Create PyTorch datasets and data loaders
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # Training loop
        num_epochs = 100
        early_stopping_epochs = 10
        best_val_loss = float('inf')
        early_stopping_counter = 0
        for epoch in range(num_epochs):
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs.unsqueeze(1))
                loss = criterion(outputs, labels.long())
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs.unsqueeze(1))
                    val_loss += criterion(outputs, labels.long()).item()
            val_loss /= len(test_loader)

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                early_stopping_counter = 0
            else:
                early_stopping_counter += 1
                if early_stopping_counter >= early_stopping_epochs:
                    break

        # Evaluation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs.unsqueeze(1))
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.long()).sum().item()
        accuracy = correct / total
        accuracy_scores.append(accuracy)

    # Compute the mean accuracy for the current combination
    mean_accuracy = np.mean(accuracy_scores)

    # Check if the current combination has higher accuracy
    if mean_accuracy > best_accuracy:
        best_accuracy = mean_accuracy
        best_combination = combination

# Print the best combination and accuracy
print("Best Combination:", best_combination)
print("Best Accuracy:", best_accuracy)

Grid Search Progress: 100%|██████████| 192/192 [1:45:50<00:00, 33.07s/it]

Best Combination: (0.1, 32, 32, 7, 'avg', 0.1, 0.01, 'adam', 'fixed', 4, 'whole_PCA')
Best Accuracy: 0.7392857142857144


And for posterity's sake, I also finetuned the multilayer perceptron approach here:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random
from tqdm import tqdm
import itertools

class ExperimentConfig:
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate, num_epochs, batch_size):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        self.batch_size = batch_size


def run_experiment(config, X, y):
    # Set random seeds for reproducibility and stable results
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True

    # Encode the target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.Tensor(X_train)
    y_train_tensor = torch.Tensor(y_train).long()
    X_test_tensor = torch.Tensor(X_test)
    y_test_tensor = torch.Tensor(y_test).long()

    # Create a PyTorch DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

    # Define the MLP model
    class MLP(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim):
            super(MLP, self).__init__()
            self.fc1 = nn.Linear(input_dim, hidden_dim)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_dim, output_dim)

        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)
            x = self.fc2(x)
            return x

    # Create an instance of the MLP model
    model = MLP(config.input_dim, config.hidden_dim, config.output_dim)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)

    # Train the model with early stopping
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    best_loss = float('inf')
    best_accuracy = 0.0
    patience = 5
    counter = 0

    for epoch in range(config.num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_dataloader)

        if epoch_loss < best_loss:
            best_loss = epoch_loss
            counter = 0
            # Evaluate the model on the test set
            model.eval()
            with torch.no_grad():
                X_test_tensor = X_test_tensor.to(device)
                y_test_tensor = y_test_tensor.to(device)
                outputs = model(X_test_tensor)
                _, predicted = torch.max(outputs, 1)
                accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)
                best_accuracy = accuracy
        else:
            counter += 1
            if counter >= patience:
                break

    return best_loss, best_accuracy


# Define the hyperparameters and their values
hyperparameters = {
    'learning_rate': [0.1],
    'hidden_size': [32],
    'batch_size': [16, 32],
    'dropout_rate': [0.1],
    'weight_decay': [0.01],
    'optimizer': ['adam', 'sgd'],
    'learning_rate_schedule': ['fixed', 'step_decay'],
    'network_depth': [2, 4],
    'X_choice': ['vanilla', 'individual_PCA', 'whole_PCA']
}

# Perform grid search
best_loss = float('inf')
best_accuracy = 0.0
best_hyperparameters = None

total_combinations = np.prod([len(values) for values in hyperparameters.values()])
pbar = tqdm(total=total_combinations, desc="Grid Search")
for values in itertools.product(*hyperparameters.values()):
    if values[8] == 'vanilla':
        X = X_vanilla
    elif values[8] == 'individual_PCA':
        X = X_individual_PCA
    elif values[8] == 'whole_PCA':
        X = X_whole_PCA

    config = ExperimentConfig(
        input_dim=X.shape[1],
        hidden_dim=values[1],
        output_dim=2,
        learning_rate=values[0],
        num_epochs=100,
        batch_size=values[2]
    )

    # init target
    y = data_multiomics["Sex"].map({"Female": 0, "Male": 1}).to_numpy()

    config.dropout_rate = values[3]
    config.weight_decay = values[4]
    config.optimizer = values[5]
    config.learning_rate_schedule = values[6]
    config.network_depth = values[7]

    loss, accuracy = run_experiment(config, X, y)

    if loss < best_loss:
        best_loss = loss
        best_accuracy = accuracy
        best_hyperparameters = config

    pbar.update(1)

pbar.close()

# Print the best hyperparameters, loss, and accuracy
print("Best Hyperparameters:")
print(best_hyperparameters.__dict__)
print("Best Loss: {:.4f}".format(best_loss))
print("Best Accuracy: {:.2%}".format(best_accuracy))

Grid Search: 100%|██████████| 48/48 [00:34<00:00,  1.41it/s]

Best Hyperparameters:
{'input_dim': 51, 'hidden_dim': 32, 'output_dim': 2, 'learning_rate': 0.1, 'num_epochs': 100, 'batch_size': 16, 'dropout_rate': 0.1, 'weight_decay': 0.01, 'optimizer': 'adam', 'learning_rate_schedule': 'fixed', 'network_depth': 2}
Best Loss: 0.0000
Best Accuracy: 85.71%


Which scores higher. Seems like my assumption that a CNN would do well doesn't hold up under these assumptions.

Additionally, and more importantly, I reimplemented the models from the "Sex" prediction task which was a binary classification task and the classes were equally distributed. So Accuracy made sense. However, for this one, i.e. "gestational age", it doesn't really make sense because the classes were imbalanced (which is why I was thinking of using stratified bootstrapping. So F1 scores would have made more sense for training and validation.

Point to note: The PCA on the whole thing combined was the best scoring input selection which might be because the networks are relatively shallow. Although I'm not certain how I'd train deep networks for tasks like these with so many features. Not to mention the problems sparsity would bring to the table.

Finally for now, I'd have liked to work out the relative importance of the features, but I'm running into serious compute limitations on Colab. So I'll leave the current experimentation with the statement:
"I have a truly marvelous demonstration of this proposition that this margin is too narrow to contain."